In [1]:
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import time
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from scipy import stats
import statistics as stat
from sklearn.model_selection import KFold

In [2]:
ds=pd.read_csv("TrainingDataset.arff.txt",header=None, comment='@')


In [3]:
#feature selection (27/30)

corr= ds.corr().abs()
columns= np.full((corr.shape[0],),True, dtype=bool)
for i in range(corr.shape[0]):
   
    for j in range(i+1, corr.shape[0]-1):
        if corr.iloc[i,j]>=0.9:
            if columns[j] and columns[i]:
                if corr.iloc[i,corr.shape[0]-1]>corr.iloc[j,corr.shape[0]-1]:
                    columns[j]=False
                else:
                    columns[i]=False
                    
#elimina la 22esima feature(indice 21)

for i in range(0,ds.shape[1]-1):
    variance = np.var(ds.iloc[:,i])
    #selezione feature in base alla varianza
    if(variance<0.2):
        columns[i]=False
        
selected_columns=ds.columns[columns]
ds= ds[selected_columns]   
print(np.shape(ds))

(11055, 28)


In [4]:
start_time= time.time()

C_range = np.logspace(-3 , 3, num=7)
Gamma_range = np.logspace(-3 , 3, num=7)
parameters = {'C':C_range, 'gamma':Gamma_range,'kernel':['rbf']}

train_scores_matrix = []
validation_scores_matrix = []
test_scores_matrix = []
best_estimators=[]
results=dict()

outer=KFold(n_splits=10)
outer.get_n_splits(ds)
count=0
for train_index, test_index in outer.split(ds):
    count+=1
    elapsed_time= time.time()
    print("\nIterazione",count)
    print("train+validation:", train_index, "test:", test_index)
    train_scores_c_perIter = []
    validation_scores_c_perIter = []
    test_scores_c_perIter = []
    
    X_train, X_test= ds.iloc[train_index,:np.shape(ds)[1]-1], ds.iloc[test_index,:np.shape(ds)[1]-1]
    Y_train, Y_test= ds.iloc[train_index,-1], ds.iloc[test_index,-1]
    #GridSearch con i parametri definiti sopra
    svc = svm.SVC()
    clf = GridSearchCV(svc, parameters, cv=10, scoring="accuracy",n_jobs=-1, return_train_score=True)
    clf.fit(X_train, Y_train)
    best_estimators.append(clf.best_params_)
    
    for i in range(0,len(clf.cv_results_['params'])):
        print ("Modello:",clf.cv_results_['params'][i], "accuracy:",clf.cv_results_['mean_test_score'][i])
    
    t_score= clf.score(X_test, Y_test)
    print ("Miglior modello:",clf.best_params_,"con score sul test set:",t_score)

    if str(clf.best_params_) in results:
        results[str(clf.best_params_)]['occurrency']+=1
        results[str(clf.best_params_)]['tests'].append(t_score)
    else:
        results[str(clf.best_params_)]=dict()
        results[str(clf.best_params_)]['occurrency']=1
        results[str(clf.best_params_)]['tests']=[]
        results[str(clf.best_params_)]['tests'].append(t_score)
    elapsed_time=time.time()-elapsed_time
    print("\nElapsed time:",elapsed_time)

'''
    # Draw heatmap of the validation accuracy as a function of gamma and C
    # The score are encoded as colors with the hot colormap which varies from dark
    # red to bright yellow. As the most interesting scores are all located in the
    # 0.92 to 0.97 range we use a custom normalizer to set the mid-point to 0.92 so
    # as to make it easier to visualize the small variations of score values in the
    # interesting range while not brutally collapsing all the low score values to
    # the same color.
    scores = clf.cv_results_['mean_test_score'].reshape(len(C_range),
                                                     len(Gamma_range))

    plt.figure(figsize=(5, 3))
    plt.subplots_adjust(left=.2, right=0.95, bottom=0.15, top=0.95)
    plt.imshow(scores, interpolation='nearest', cmap=plt.cm.hot,
               norm=MidpointNormalize(vmin=0.2, midpoint=0.92))
    plt.xlabel('gamma')
    plt.ylabel('C')
    plt.colorbar()
    plt.xticks(np.arange(len(Gamma_range)), Gamma_range, rotation=45)
    plt.yticks(np.arange(len(C_range)), C_range)
    plt.title('Validation accuracy')
    plt.show()
    
  
    
    ################    
    i=0
    maxdim=len(Gamma_range)
    while(i<len(clf.cv_results_['params'])):
        
        c=clf.cv_results_['params'][i]['C']

        train_scores_gamma_perC=[]
        validation_scores_gamma_perC=[]
        test_scores_gamma_perC=[]
        for j in range(i, i+maxdim):
            gamma=clf.cv_results_['params'][j]['gamma']
            validation_scores_gamma_perC.append(clf.cv_results_['mean_test_score'][j])  
            train_scores_gamma_perC.append(clf.cv_results_['mean_train_score'][j])
            
            temp_class = svm.SVC(kernel='rbf', C=c, gamma=gamma)
            temp_class.fit(X_train,Y_train)
            #n risultati sul test set 
            test_scores_gamma_perC.append(temp_class.score(X_test,Y_test))
        
        validation_scores_c_perIter.append(validation_scores_gamma_perC)
        train_scores_c_perIter.append(train_scores_gamma_perC)
        test_scores_c_perIter.append(test_scores_gamma_perC)

        i=i+maxdim
    #####################
    
    #train validation e test scores per ogni shuffle
    train_scores_matrix.append(train_scores_c_perIter)
    validation_scores_matrix.append(validation_scores_c_perIter)
    test_scores_matrix.append(test_scores_c_perIter)
   ''' 
    


elapsed_time=time.time()-start_time
print("\nTotal elapsed time:",elapsed_time)


Iterazione 1
train+validation: [ 1106  1107  1108 ... 11052 11053 11054] test: [   0    1    2 ... 1103 1104 1105]
Modello: {'C': 0.001, 'gamma': 0.001, 'kernel': 'rbf'} accuracy: 0.5570409086340335
Modello: {'C': 0.001, 'gamma': 0.01, 'kernel': 'rbf'} accuracy: 0.5570409086340335
Modello: {'C': 0.001, 'gamma': 0.1, 'kernel': 'rbf'} accuracy: 0.5570409086340335
Modello: {'C': 0.001, 'gamma': 1.0, 'kernel': 'rbf'} accuracy: 0.5570409086340335
Modello: {'C': 0.001, 'gamma': 10.0, 'kernel': 'rbf'} accuracy: 0.5570409086340335
Modello: {'C': 0.001, 'gamma': 100.0, 'kernel': 'rbf'} accuracy: 0.5570409086340335
Modello: {'C': 0.001, 'gamma': 1000.0, 'kernel': 'rbf'} accuracy: 0.5570409086340335
Modello: {'C': 0.01, 'gamma': 0.001, 'kernel': 'rbf'} accuracy: 0.5570409086340335
Modello: {'C': 0.01, 'gamma': 0.01, 'kernel': 'rbf'} accuracy: 0.9060207055985526
Modello: {'C': 0.01, 'gamma': 0.1, 'kernel': 'rbf'} accuracy: 0.8365664890943814
Modello: {'C': 0.01, 'gamma': 1.0, 'kernel': 'rbf'} acc

C:\Users\Chiara\Anaconda3\lib\site-packages\sklearn\externals\joblib\externals\loky\process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Modello: {'C': 0.001, 'gamma': 0.001, 'kernel': 'rbf'} accuracy: 0.5560357824907026
Modello: {'C': 0.001, 'gamma': 0.01, 'kernel': 'rbf'} accuracy: 0.5560357824907026
Modello: {'C': 0.001, 'gamma': 0.1, 'kernel': 'rbf'} accuracy: 0.5560357824907026
Modello: {'C': 0.001, 'gamma': 1.0, 'kernel': 'rbf'} accuracy: 0.5560357824907026
Modello: {'C': 0.001, 'gamma': 10.0, 'kernel': 'rbf'} accuracy: 0.5560357824907026
Modello: {'C': 0.001, 'gamma': 100.0, 'kernel': 'rbf'} accuracy: 0.5560357824907026
Modello: {'C': 0.001, 'gamma': 1000.0, 'kernel': 'rbf'} accuracy: 0.5560357824907026
Modello: {'C': 0.01, 'gamma': 0.001, 'kernel': 'rbf'} accuracy: 0.5560357824907026
Modello: {'C': 0.01, 'gamma': 0.01, 'kernel': 'rbf'} accuracy: 0.9058196803698865
Modello: {'C': 0.01, 'gamma': 0.1, 'kernel': 'rbf'} accuracy: 0.836264951251382
Modello: {'C': 0.01, 'gamma': 1.0, 'kernel': 'rbf'} accuracy: 0.5560357824907026
Modello: {'C': 0.01, 'gamma': 10.0, 'kernel': 'rbf'} accuracy: 0.5560357824907026
Modello: 

C:\Users\Chiara\Anaconda3\lib\site-packages\sklearn\externals\joblib\externals\loky\process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Modello: {'C': 0.001, 'gamma': 0.001, 'kernel': 'rbf'} accuracy: 0.5583475726203638
Modello: {'C': 0.001, 'gamma': 0.01, 'kernel': 'rbf'} accuracy: 0.5583475726203638
Modello: {'C': 0.001, 'gamma': 0.1, 'kernel': 'rbf'} accuracy: 0.5583475726203638
Modello: {'C': 0.001, 'gamma': 1.0, 'kernel': 'rbf'} accuracy: 0.5583475726203638
Modello: {'C': 0.001, 'gamma': 10.0, 'kernel': 'rbf'} accuracy: 0.5583475726203638
Modello: {'C': 0.001, 'gamma': 100.0, 'kernel': 'rbf'} accuracy: 0.5583475726203638
Modello: {'C': 0.001, 'gamma': 1000.0, 'kernel': 'rbf'} accuracy: 0.5583475726203638
Modello: {'C': 0.01, 'gamma': 0.001, 'kernel': 'rbf'} accuracy: 0.5583475726203638
Modello: {'C': 0.01, 'gamma': 0.01, 'kernel': 'rbf'} accuracy: 0.905317117298221
Modello: {'C': 0.01, 'gamma': 0.1, 'kernel': 'rbf'} accuracy: 0.8368680269373806
Modello: {'C': 0.01, 'gamma': 1.0, 'kernel': 'rbf'} accuracy: 0.5583475726203638
Modello: {'C': 0.01, 'gamma': 10.0, 'kernel': 'rbf'} accuracy: 0.5583475726203638
Modello: 

...

Modello: {'C': 0.001, 'gamma': 0.001, 'kernel': 'rbf'} accuracy: 0.556180904522613
Modello: {'C': 0.001, 'gamma': 0.01, 'kernel': 'rbf'} accuracy: 0.556180904522613
Modello: {'C': 0.001, 'gamma': 0.1, 'kernel': 'rbf'} accuracy: 0.556180904522613
Modello: {'C': 0.001, 'gamma': 1.0, 'kernel': 'rbf'} accuracy: 0.556180904522613
Modello: {'C': 0.001, 'gamma': 10.0, 'kernel': 'rbf'} accuracy: 0.556180904522613
Modello: {'C': 0.001, 'gamma': 100.0, 'kernel': 'rbf'} accuracy: 0.556180904522613
Modello: {'C': 0.001, 'gamma': 1000.0, 'kernel': 'rbf'} accuracy: 0.556180904522613
Modello: {'C': 0.01, 'gamma': 0.001, 'kernel': 'rbf'} accuracy: 0.556180904522613
Modello: {'C': 0.01, 'gamma': 0.01, 'kernel': 'rbf'} accuracy: 0.9088442211055276
Modello: {'C': 0.01, 'gamma': 0.1, 'kernel': 'rbf'} accuracy: 0.84
Modello: {'C': 0.01, 'gamma': 1.0, 'kernel': 'rbf'} accuracy: 0.556180904522613
Modello: {'C': 0.01, 'gamma': 10.0, 'kernel': 'rbf'} accuracy: 0.556180904522613
Modello: {'C': 0.01, 'gamma': 10

C:\Users\Chiara\Anaconda3\lib\site-packages\sklearn\externals\joblib\externals\loky\process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Modello: {'C': 0.001, 'gamma': 0.001, 'kernel': 'rbf'} accuracy: 0.5576884422110553
Modello: {'C': 0.001, 'gamma': 0.01, 'kernel': 'rbf'} accuracy: 0.5576884422110553
Modello: {'C': 0.001, 'gamma': 0.1, 'kernel': 'rbf'} accuracy: 0.5576884422110553
Modello: {'C': 0.001, 'gamma': 1.0, 'kernel': 'rbf'} accuracy: 0.5576884422110553
Modello: {'C': 0.001, 'gamma': 10.0, 'kernel': 'rbf'} accuracy: 0.5576884422110553
Modello: {'C': 0.001, 'gamma': 100.0, 'kernel': 'rbf'} accuracy: 0.5576884422110553
Modello: {'C': 0.001, 'gamma': 1000.0, 'kernel': 'rbf'} accuracy: 0.5576884422110553
Modello: {'C': 0.01, 'gamma': 0.001, 'kernel': 'rbf'} accuracy: 0.5576884422110553
Modello: {'C': 0.01, 'gamma': 0.01, 'kernel': 'rbf'} accuracy: 0.9044221105527638
Modello: {'C': 0.01, 'gamma': 0.1, 'kernel': 'rbf'} accuracy: 0.8332663316582914
Modello: {'C': 0.01, 'gamma': 1.0, 'kernel': 'rbf'} accuracy: 0.5576884422110553
Modello: {'C': 0.01, 'gamma': 10.0, 'kernel': 'rbf'} accuracy: 0.5576884422110553
Modello:

C:\Users\Chiara\Anaconda3\lib\site-packages\sklearn\externals\joblib\externals\loky\process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Modello: {'C': 0.001, 'gamma': 0.001, 'kernel': 'rbf'} accuracy: 0.5562814070351759
Modello: {'C': 0.001, 'gamma': 0.01, 'kernel': 'rbf'} accuracy: 0.5562814070351759
Modello: {'C': 0.001, 'gamma': 0.1, 'kernel': 'rbf'} accuracy: 0.5562814070351759
Modello: {'C': 0.001, 'gamma': 1.0, 'kernel': 'rbf'} accuracy: 0.5562814070351759
Modello: {'C': 0.001, 'gamma': 10.0, 'kernel': 'rbf'} accuracy: 0.5562814070351759
Modello: {'C': 0.001, 'gamma': 100.0, 'kernel': 'rbf'} accuracy: 0.5562814070351759
Modello: {'C': 0.001, 'gamma': 1000.0, 'kernel': 'rbf'} accuracy: 0.5562814070351759
Modello: {'C': 0.01, 'gamma': 0.001, 'kernel': 'rbf'} accuracy: 0.5562814070351759
Modello: {'C': 0.01, 'gamma': 0.01, 'kernel': 'rbf'} accuracy: 0.9042211055276382
Modello: {'C': 0.01, 'gamma': 0.1, 'kernel': 'rbf'} accuracy: 0.8374874371859297
Modello: {'C': 0.01, 'gamma': 1.0, 'kernel': 'rbf'} accuracy: 0.5562814070351759
Modello: {'C': 0.01, 'gamma': 10.0, 'kernel': 'rbf'} accuracy: 0.5562814070351759
Modello:

In [5]:
print(results,"\n")
flag=False
duplicates=[]
tot=0
for x in results:
    tot=tot+results[x]['occurrency']
occ=0
best=None
for key in results:
    if results[key]['occurrency']>occ:
        occ=results[key]['occurrency']
        best=key
        testscores=results[key]['tests']
    elif results[key]['occurrency']==occ:
        flag=True
        duplicates.append(key)
print(best,"occorre",occ,"volte su",tot,".\nMedia dei test score:",np.mean(testscores), "varianza:", np.var(testscores))
if flag:
    print("Ci sono più modelli con la stessa occorrenza:")
    for x in duplicates:
        print(x,"occorre",results[x]['occurrency'],"volte su",tot,".\nMedia dei test score:",np.mean(results[x]['tests']), "varianza:", np.var(results[x]['tests']))

{"{'C': 10.0, 'gamma': 0.1, 'kernel': 'rbf'}": {'occurrency': 7, 'tests': [0.9773960216998192, 0.9810126582278481, 0.9719710669077758, 0.9819168173598554, 0.9792043399638336, 0.9728506787330317, 0.96289592760181]}, "{'C': 100.0, 'gamma': 0.1, 'kernel': 'rbf'}": {'occurrency': 2, 'tests': [0.9547511312217195, 0.9583710407239819]}, "{'C': 1000.0, 'gamma': 0.1, 'kernel': 'rbf'}": {'occurrency': 1, 'tests': [0.9484162895927601]}} 

{'C': 10.0, 'gamma': 0.1, 'kernel': 'rbf'} occorre 7 volte su 10 .
Media dei test score: 0.9753210729277105 varianza: 3.814182675772537e-05


In [6]:
start_time= time.time()

C_range = np.logspace(-3 , 3, num=7)
parameters = {'C':C_range, 'kernel':['linear']}
train_scores_matrix = []
validation_scores_matrix = []
test_scores_matrix = []
best_estimators=[]
results=dict()

outer=KFold(n_splits=10)
outer.get_n_splits(ds)
count=0
for train_index, test_index in outer.split(ds):
    count+=1
    elapsed_time= time.time()
    print("\nIterazione",count)
    print("train+validation:", train_index, "test:", test_index)
    train_scores=[]
    validation_scores=[]
    test_scores=[]
    X_train, X_test= ds.iloc[train_index,:np.shape(ds)[1]-1], ds.iloc[test_index,:np.shape(ds)[1]-1]
    Y_train, Y_test= ds.iloc[train_index,-1], ds.iloc[test_index,-1]
    #GridSearch con i parametri definiti sopra
    svc = svm.SVC()
    clf = GridSearchCV(svc, parameters, cv=10, scoring="accuracy",n_jobs=-1, return_train_score=True)
    clf.fit(X_train, Y_train)
    best_estimators.append(clf.best_params_)
    
    for i in range(0,len(clf.cv_results_['params'])):
        print ("Modello:",clf.cv_results_['params'][i], "accuracy:",clf.cv_results_['mean_test_score'][i])
        
    t_score= clf.score(X_test, Y_test)
    print ("Miglior modello:",clf.best_params_,"con score sul test set:",t_score)

    if str(clf.best_params_) in results:
        results[str(clf.best_params_)]['occurrency']+=1
        results[str(clf.best_params_)]['tests'].append(t_score)
    else:
        results[str(clf.best_params_)]=dict()
        results[str(clf.best_params_)]['occurrency']=1
        results[str(clf.best_params_)]['tests']=[]
        results[str(clf.best_params_)]['tests'].append(t_score)
    
    '''
    #combinazioni di parametri
    hyperparameters = []
    for i in clf.cv_results_['params']:
        hyperparameters.append(i)
    
    #score sul test set per ogni combinazione
    for i in hyperparameters:
        temp_class = svm.SVC(kernel=i['kernel'],C=i['C'])
        temp_class.fit(X_train,Y_train)
        #n risultati sul test set 
        test_scores.append(temp_class.score(X_test,Y_test))
        
    for i in range (0,len(clf.cv_results_['mean_test_score'])): 
        train_scores.append(clf.cv_results_['mean_train_score'][i])
        validation_scores.append(clf.cv_results_['mean_test_score'][i])  
    
    #train validation e test scores per ogni iterazione
    train_scores_matrix.append(train_scores)
    validation_scores_matrix.append(validation_scores)
    test_scores_matrix.append(test_scores)
    '''
    elapsed_time=time.time()-elapsed_time
    print("\nElapsed time:",elapsed_time)


elapsed_time=time.time()-start_time
print("\nTotal elapsed time:",elapsed_time)


Iterazione 1
train+validation: [ 1106  1107  1108 ... 11052 11053 11054] test: [   0    1    2 ... 1103 1104 1105]
Modello: {'C': 0.001, 'kernel': 'linear'} accuracy: 0.9151673535028646
Modello: {'C': 0.01, 'kernel': 'linear'} accuracy: 0.9222032365061815
Modello: {'C': 0.1, 'kernel': 'linear'} accuracy: 0.9231078500351794
Modello: {'C': 1.0, 'kernel': 'linear'} accuracy: 0.9233088752638456
Modello: {'C': 10.0, 'kernel': 'linear'} accuracy: 0.9230073374208463
Modello: {'C': 100.0, 'kernel': 'linear'} accuracy: 0.9230073374208463
Modello: {'C': 1000.0, 'kernel': 'linear'} accuracy: 0.9228063121921801
Miglior modello: {'C': 1.0, 'kernel': 'linear'} con score sul test set: 0.9204339963833634

Elapsed time: 648.7884156703949

Iterazione 2
train+validation: [    0     1     2 ... 11052 11053 11054] test: [1106 1107 1108 ... 2209 2210 2211]
Modello: {'C': 0.001, 'kernel': 'linear'} accuracy: 0.9146647904311991
Modello: {'C': 0.01, 'kernel': 'linear'} accuracy: 0.9224042617348477
Modello: {'

In [7]:
print(results,"\n")
flag=False
duplicates=[]
tot=0
for x in results:
    tot=tot+results[x]['occurrency']
occ=0
best=None
for key in results:
    if results[key]['occurrency']>occ:
        occ=results[key]['occurrency']
        best=key
        testscores=results[key]['tests']
    elif results[key]['occurrency']==occ:
        flag=True
        duplicates.append(key)
print(best,"occorre",occ,"volte su",tot,".\nMedia dei test score:",np.mean(testscores),"varianza:",np.var(testscores))
if flag:
    print("Ci sono più modelli con la stessa occorrenza:")
    for x in duplicates:
        print(x,"occorre",results[x]['occurrency'],"volte su",tot,".\nMedia dei test score:",np.mean(results[x]['tests']),"varianza:",np.var(results[x]['tests']))

{"{'C': 1.0, 'kernel': 'linear'}": {'occurrency': 3, 'tests': [0.9204339963833634, 0.9366515837104072, 0.9294117647058824]}, "{'C': 0.1, 'kernel': 'linear'}": {'occurrency': 2, 'tests': [0.9222423146473779, 0.9248868778280543]}, "{'C': 10.0, 'kernel': 'linear'}": {'occurrency': 2, 'tests': [0.9258589511754068, 0.918625678119349]}, "{'C': 1000.0, 'kernel': 'linear'}": {'occurrency': 1, 'tests': [0.9285714285714286]}, "{'C': 100.0, 'kernel': 'linear'}": {'occurrency': 2, 'tests': [0.918552036199095, 0.9104072398190045]}} 

{'C': 1.0, 'kernel': 'linear'} occorre 3 volte su 10 .
Media dei test score: 0.928832448266551 varianza: 4.400282688682181e-05
